In [74]:
from pyspark.sql import SparkSession
import sys
spark = SparkSession.builder.getOrCreate()
from pymongo import MongoClient
import hdfs
from pywebhdfs.webhdfs import PyWebHdfsClient
from pprint import pprint
import json
from datetime import datetime as dt
import datetime
import pandas as pd
import pytz

In [79]:
hdfs = PyWebHdfsClient(host='hadoop-master',port='9870', user_name='kafka-data')  # your Namenode IP & username here
my_dir = '/user/kafka-data'
data_files = [i["pathSuffix"] for i in hdfs.list_dir(my_dir)["FileStatuses"]["FileStatus"]]
pprint([data_files, len(data_files), len(datafiles)*50], compact=True)

NameError: name 'datafiles' is not defined

In [76]:
conn = MongoClient('mongo', 27017)
db = conn.tweets
collection = db.tweet

In [77]:
collection.drop()

for i in data_files:
    df = spark.read.json(f'hdfs://hadoop-master:9000/user/kafka-data/{i}')
    json_data = df.toJSON().collect()
    for i in json_data:
        data = json.loads(i)
        hashtags = [i for i in data["hashtags"]]
        for h in hashtags:
            hours = 2
            hours_added = datetime.timedelta(hours = hours)
            new_datetime = dt.fromtimestamp(int(data["timestamp"])/1000)
            new_datetime = new_datetime + hours_added

            uniqued_hashtag = {"hashtag": h, "datetime": new_datetime}
            collection.insert_one(uniqued_hashtag)
        
cursor = collection.find()
for record in cursor:
    print(record)

{'_id': ObjectId('60d5b0d5e0a097dedfdaa52c'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 39, 878000)}
{'_id': ObjectId('60d5b0d5e0a097dedfdaa52d'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 86000)}
{'_id': ObjectId('60d5b0d5e0a097dedfdaa52e'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 374000)}
{'_id': ObjectId('60d5b0d5e0a097dedfdaa52f'), 'hashtag': 'NazisrausausdenStadien', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5b0d5e0a097dedfdaa530'), 'hashtag': 'HUNGER', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5b0d5e0a097dedfdaa531'), 'hashtag': 'Goretzka', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5b0d5e0a097dedfdaa532'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5b0d5e0a097dedfdaa533'), 'hashtag': '

{'_id': ObjectId('60d5b0d6e0a097dedfdaaab4'), 'hashtag': 'Heineken', 'datetime': datetime.datetime(2021, 6, 25, 11, 38, 9, 611000)}
{'_id': ObjectId('60d5b0d6e0a097dedfdaaab5'), 'hashtag': 'musulmane', 'datetime': datetime.datetime(2021, 6, 25, 11, 38, 9, 611000)}
{'_id': ObjectId('60d5b0d6e0a097dedfdaaab6'), 'hashtag': 'ManCity', 'datetime': datetime.datetime(2021, 6, 25, 11, 38, 10, 209000)}
{'_id': ObjectId('60d5b0d6e0a097dedfdaaab7'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 38, 10, 209000)}
{'_id': ObjectId('60d5b0d6e0a097dedfdaaab8'), 'hashtag': 'Euro2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 38, 10, 370000)}
{'_id': ObjectId('60d5b0d6e0a097dedfdaaab9'), 'hashtag': 'HBRFanzone', 'datetime': datetime.datetime(2021, 6, 25, 11, 38, 10, 370000)}
{'_id': ObjectId('60d5b0d6e0a097dedfdaaaba'), 'hashtag': 'Last16', 'datetime': datetime.datetime(2021, 6, 25, 11, 38, 10, 370000)}
{'_id': ObjectId('60d5b0d6e0a097dedfdaaabb'), 'hashtag': 'EURO2020', 'd

{'_id': ObjectId('60d5b0d7e0a097dedfdab075'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 46, 17, 825000)}
{'_id': ObjectId('60d5b0d7e0a097dedfdab076'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 46, 18, 530000)}
{'_id': ObjectId('60d5b0d7e0a097dedfdab077'), 'hashtag': 'WAL', 'datetime': datetime.datetime(2021, 6, 25, 11, 46, 18, 707000)}
{'_id': ObjectId('60d5b0d7e0a097dedfdab078'), 'hashtag': 'den', 'datetime': datetime.datetime(2021, 6, 25, 11, 46, 18, 707000)}
{'_id': ObjectId('60d5b0d7e0a097dedfdab079'), 'hashtag': 'ITA', 'datetime': datetime.datetime(2021, 6, 25, 11, 46, 18, 707000)}
{'_id': ObjectId('60d5b0d7e0a097dedfdab07a'), 'hashtag': 'AUT', 'datetime': datetime.datetime(2021, 6, 25, 11, 46, 18, 707000)}
{'_id': ObjectId('60d5b0d7e0a097dedfdab07b'), 'hashtag': 'NED', 'datetime': datetime.datetime(2021, 6, 25, 11, 46, 18, 707000)}
{'_id': ObjectId('60d5b0d7e0a097dedfdab07c'), 'hashtag': 'CZE', 'datetime': datetime.datetime(

{'_id': ObjectId('60d5b0d8e0a097dedfdab6a9'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 56, 35, 188000)}
{'_id': ObjectId('60d5b0d8e0a097dedfdab6aa'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 56, 35, 472000)}
{'_id': ObjectId('60d5b0d8e0a097dedfdab6ab'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 56, 36, 721000)}
{'_id': ObjectId('60d5b0d8e0a097dedfdab6ac'), 'hashtag': 'Heineken', 'datetime': datetime.datetime(2021, 6, 25, 11, 56, 36, 721000)}
{'_id': ObjectId('60d5b0d8e0a097dedfdab6ad'), 'hashtag': 'WAL', 'datetime': datetime.datetime(2021, 6, 25, 11, 56, 37, 699000)}
{'_id': ObjectId('60d5b0d8e0a097dedfdab6ae'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 56, 37, 699000)}
{'_id': ObjectId('60d5b0d8e0a097dedfdab6af'), 'hashtag': 'TogetherStronger', 'datetime': datetime.datetime(2021, 6, 25, 11, 56, 37, 699000)}
{'_id': ObjectId('60d5b0d8e0a097dedfdab6b0'), 'hashtag': 'EURO2020

{'_id': ObjectId('60d5b0d9e0a097dedfdabc95'), 'hashtag': 'XoliMfeka', 'datetime': datetime.datetime(2021, 6, 25, 12, 13, 38, 179000)}
{'_id': ObjectId('60d5b0d9e0a097dedfdabc96'), 'hashtag': 'ENG', 'datetime': datetime.datetime(2021, 6, 25, 12, 13, 40, 571000)}
{'_id': ObjectId('60d5b0d9e0a097dedfdabc97'), 'hashtag': 'GER', 'datetime': datetime.datetime(2021, 6, 25, 12, 13, 40, 571000)}
{'_id': ObjectId('60d5b0d9e0a097dedfdabc98'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 12, 13, 42, 313000)}
{'_id': ObjectId('60d5b0d9e0a097dedfdabc99'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 12, 13, 42, 585000)}
{'_id': ObjectId('60d5b0d9e0a097dedfdabc9a'), 'hashtag': 'VIVOY72', 'datetime': datetime.datetime(2021, 6, 25, 12, 13, 42, 585000)}
{'_id': ObjectId('60d5b0d9e0a097dedfdabc9b'), 'hashtag': 'ChampsElysees', 'datetime': datetime.datetime(2021, 6, 25, 12, 13, 43, 134000)}
{'_id': ObjectId('60d5b0d9e0a097dedfdabc9c'), 'hashtag': 'Paris', 'datetim